## install libraries

In [9]:
!pip show datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: datasets
Version: 4.2.0
Summary: HuggingFace community-driven open-source library of datasets
Home-page: https://github.com/huggingface/datasets
Author: HuggingFace Inc.
Author-email: thomas@huggingface.co
License: Apache 2.0
Location: /workspace/miniconda3/envs/chatbot-v3/lib/python3.12/site-packages
Requires: dill, filelock, fsspec, httpx, huggingface-hub, multiprocess, numpy, packaging, pandas, pyarrow, pyyaml, requests, tqdm, xxhash
Required-by: 


In [7]:
!pip install -U datasets 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached datasets-4.2.0-py3-none-any.whl.metadata (18 kB)
  Using cached pyarrow-21.0.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached anyio-4.11.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
Using cached datasets-4.2.0-py3-none-any.whl (506 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
Using cached h11-0.16.0-py3-none-any.whl (37 kB)
Using cached pyarrow-21.0.0-cp312-cp312-manylinux_2_28_x86_64.whl (42.8 MB)
Using cached anyio-4.11.0-py3-none-any.whl (109 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.0
    Uninstalling pyarrow-19.0.0:━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

## load data

In [ ]:
from huggingface_hub import login


/workspace/miniconda3/envs/chatbot-v3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset
corpus = load_dataset("batmangiaicuuthegioi/gtdb-corpus")
gtdb_dataset = load_dataset("batmangiaicuuthegioi/giao-thong-duong-bo")
hard_negatives = load_dataset("batmangiaicuuthegioi/hard-negatives-traffic-law")

In [9]:
corpus['train'][0]

{'id': '36/2024/QH15/1',
 'text': '[36/2024/QH15] Điều 1\n\nĐiều 1. Phạm vi điều chỉnh\n\nLuật này quy định về quy tắc, phương tiện, người tham gia giao thông đường bộ, chỉ huy, điều khiển, tuần tra, kiểm soát, giải quyết tai nạn giao thông đường bộ, trách nhiệm quản lý nhà nước và trách nhiệm của cơ quan, tổ chức, cá nhân có liên quan đến trật tự, an toàn giao thông đường bộ.'}

In [8]:
gtdb_dataset['train'][0]

{'question': 'Trật tự, an toàn giao thông đường bộ được hiểu là gì?',
 'answer': 'Trật tự, an toàn giao thông đường bộ là trạng thái giao thông trên đường bộ có trật tự, bảo đảm an toàn, thông suốt; được hình thành và điều chỉnh bởi các quy tắc, nguyên tắc, quy phạm pháp luật trong lĩnh vực giao thông đường bộ (Điều 2 Mục 1 - 36/2024/QH15).',
 'question_type': 'Câu hỏi phân biệt/khái niệm',
 'answer_id': '36/2024/QH15/2/1'}

In [10]:
hard_negatives['train'][0]

{'question': 'Tình huống nào liên quan đến chướng ngại vật trên đường đòi hỏi sự cẩn trọng của người lái xe?',
 'answer': 'Khi trên đường xuất hiện chướng ngại vật, đây là một tình huống đòi hỏi người lái xe phải cẩn trọng để xử lý. Tình huống này được liệt kê là một trong những trường hợp đặc biệt cần lưu ý khi tham gia giao thông. (Theo Điểm b Khoản 3 Điều 12 [36/2024/QH15/12/3/b])',
 'negative': 'Khi trên đường có chướng ngại vật, đây được mô tả là một tình huống giao thông cần phải chú ý. Sự xuất hiện của vật cản bất ngờ đòi hỏi người lái xe phải có sự chuẩn bị và phản ứng kịp thời. (Theo Điểm b Khoản 3 Điều 12 [36/2024/QH15/12/3/b])'}

## load model

In [3]:
import numpy as np
from sentence_transformers import SentenceTransformer, util

pretrained_model = SentenceTransformer('BAAI/bge-m3', device='cuda')
best_model = SentenceTransformer('AITeamVN/Vietnamese_Embedding', device='cuda')
finetuned_model = SentenceTransformer('batmangiaicuuthegioi/GTDB-EMBEDDING_MODEL-1epoch_1e-6', device='cuda')

In [4]:
print("Pretrained model:", pretrained_model.encode("Xin chào thế giới"))

Pretrained model: [-0.03093209  0.03995155 -0.03753064 ...  0.02753271 -0.03306824
  0.00220284]


## inference and test (with the last 100 questions)

In [5]:
corpus = corpus['train']
dataset = gtdb_dataset['train']

In [22]:
# write corpus to a jsonl file
import json
with open("corpus.jsonl", "w", encoding="utf-8") as f:
    for item in corpus:
        f.write(json.dumps(item) + "\n")

In [ ]:
dataset[100]

{'question': 'Trật tự, an toàn giao thông đường bộ được hiểu là gì?',
 'answer': 'Trật tự, an toàn giao thông đường bộ là trạng thái giao thông trên đường bộ có trật tự, bảo đảm an toàn, thông suốt; được hình thành và điều chỉnh bởi các quy tắc, nguyên tắc, quy phạm pháp luật trong lĩnh vực giao thông đường bộ (Điều 2 Mục 1 - 36/2024/QH15).',
 'question_type': 'Câu hỏi phân biệt/khái niệm',
 'answer_id': '36/2024/QH15/2/1'}

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer, util

embedding_model = best_model  # Hoặc best_model, pretrained_model tùy chọn

test_samples = dataset[-100:]

# Encode toàn bộ corpus trước để tránh lặp lại
corpus_texts = [c["text"] for c in corpus]
corpus_ids = [c["id"] for c in corpus]


Batches:   0%|          | 0/22 [00:00<?, ?it/s]

Batches: 100%|██████████| 22/22 [00:02<00:00,  7.98it/s]


In [23]:
len(corpus_embeds)

1390

In [ ]:
type(test_samples['answer'][990]

'Đối với xe máy chuyên dùng, pháp luật quy định về giới hạn kích thước và giới hạn tải trọng. Khi tham gia giao thông, các loại xe này không được vượt quá các giới hạn đã được cơ quan có thẩm quyền ban hành. (Theo Điểm g Khoản 5 Điều 42 [36/2024/QH15/42/5/g])'

In [42]:
len(test_samples['answer'])

100

In [44]:
def evaluate_retrieval(embedding_model):
    corpus_embeds = embedding_model.encode(corpus_texts, batch_size=64, convert_to_tensor=True, show_progress_bar=True, normalize_embeddings=True)
    R1, R5 = 0, 0  # Biến đếm recall@1 và recall@5

    for question, answer_id in zip(test_samples['question'], test_samples['answer_id']):
        q_emb = embedding_model.encode(question, convert_to_tensor=True, normalize_embeddings=True)
        # Tính similarity cosine
        cos_scores = util.cos_sim(q_emb, corpus_embeds)[0].cpu().numpy()
        # Lấy top 5 kết quả
        top_k = np.argsort(-cos_scores)[:5]
        top_k_ids = [corpus_ids[i] for i in top_k]
        # print(top_k_ids)

        gt_id = answer_id.split("/")[0] + "/" + answer_id.split("/")[1]  # hoặc tùy theo format id bạn muốn so khớp
        # Một số bạn dùng match theo prefix "36/2024/QH15" thay vì full path

        if answer_id == top_k_ids[0]:
            R1 += 1
        if answer_id in top_k_ids:
            R5 += 1

    # Tính tỷ lệ recall
    R1_score = R1 / len(test_samples['question'])
    R5_score = R5 / len(test_samples['question'])

    print(f"R@1 = {R1_score:.3f}, R@5 = {R5_score:.3f}")
    return R1, R5, len(test_samples)
    
evaluate_retrieval(best_model)
evaluate_retrieval(pretrained_model)
evaluate_retrieval(finetuned_model)

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

Batches: 100%|██████████| 22/22 [00:02<00:00,  8.00it/s]


R@1 = 0.540, R@5 = 0.830


Batches: 100%|██████████| 22/22 [00:02<00:00,  7.99it/s]


R@1 = 0.520, R@5 = 0.770


Batches: 100%|██████████| 22/22 [00:02<00:00,  8.01it/s]


R@1 = 0.600, R@5 = 0.840


(60, 84, 4)

In [29]:
print(R1, R5, len)

647 839 4
